## Importar dependencias

In [1]:
from datetime import date

import pandas as pd
import numpy as np
import holidays
import yaml

## Conexión a las bases de datos

In [9]:
from sqlalchemy import create_engine

with open('./config.yml', 'r') as f:
        config = yaml.safe_load(f)
        config_ryf = config['RAPIDO_Y_FURIOSO']
        config_etl = config['ETL']

url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@{config_ryf['host']}:"
        f"{config_ryf['port']}/{config_ryf['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
        f"{config_etl['port']}/{config_etl['dbname']}")

ryf = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

In [4]:
dim_fecha = pd.DataFrame({
    "fecha": pd.date_range(start='1/9/2023', end='31/12/2024', freq='D')
})
dim_fecha.head()

,fecha
0,2023-01-09
1,2023-01-10
2,2023-01-11
3,2023-01-12
4,2023-01-13


In [5]:
dim_fecha['hora'] = dim_fecha['fecha'].dt.strftime('%H:%M:%S') 
dim_fecha['anio'] = dim_fecha['fecha'].dt.year
dim_fecha['mes'] = dim_fecha['fecha'].dt.month_name()
dim_fecha['dia'] = dim_fecha['fecha'].dt.day
dim_fecha['dia_semana'] = dim_fecha['fecha'].dt.weekday

dim_fecha.head()

,fecha,hora,anio,mes,dia,dia_semana
0,2023-01-09,00:00:00,2023,January,9,0
1,2023-01-10,00:00:00,2023,January,10,1
2,2023-01-11,00:00:00,2023,January,11,2
3,2023-01-12,00:00:00,2023,January,12,3
4,2023-01-13,00:00:00,2023,January,13,4


In [6]:
dim_fecha["dia_del_anio"] = dim_fecha["fecha"].dt.day_of_year
dim_fecha["dia_del_mes"] = dim_fecha["fecha"].dt.days_in_month
dim_fecha["mes_str"] = dim_fecha["fecha"].dt.month_name()
dim_fecha["dia_str"] = dim_fecha["fecha"].dt.day_name()
dim_fecha["fecha_str"] = dim_fecha["fecha"].dt.strftime("%d/%m/%Y")

dim_fecha.head()

,fecha,hora,anio,mes,dia,dia_semana,dia_del_anio,dia_del_mes,mes_str,dia_str,fecha_str
0,2023-01-09,00:00:00,2023,January,9,0,9,31,January,Monday,09/01/2023
1,2023-01-10,00:00:00,2023,January,10,1,10,31,January,Tuesday,10/01/2023
2,2023-01-11,00:00:00,2023,January,11,2,11,31,January,Wednesday,11/01/2023
3,2023-01-12,00:00:00,2023,January,12,3,12,31,January,Thursday,12/01/2023
4,2023-01-13,00:00:00,2023,January,13,4,13,31,January,Friday,13/01/2023


In [7]:
co_holidays = holidays.CO(language="es")
dim_fecha["es_festivo"] = dim_fecha["fecha"].apply(lambda x:  x in co_holidays)
dim_fecha["festivo"] = dim_fecha["fecha"].apply(lambda x: co_holidays.get(x))
dim_fecha["guardado"] = date.today()    
dim_fecha["fin_de_semana"] = dim_fecha["dia_semana"] > 4


dim_fecha.head()

,fecha,hora,anio,mes,dia,dia_semana,dia_del_anio,dia_del_mes,mes_str,dia_str,fecha_str,es_festivo,festivo,guardado,fin_de_semana
0,2023-01-09,00:00:00,2023,January,9,0,9,31,January,Monday,09/01/2023,True,Día de los Reyes Magos (observado),2024-11-10,False
1,2023-01-10,00:00:00,2023,January,10,1,10,31,January,Tuesday,10/01/2023,False,None,2024-11-10,False
2,2023-01-11,00:00:00,2023,January,11,2,11,31,January,Wednesday,11/01/2023,False,None,2024-11-10,False
3,2023-01-12,00:00:00,2023,January,12,3,12,31,January,Thursday,12/01/2023,False,None,2024-11-10,False
4,2023-01-13,00:00:00,2023,January,13,4,13,31,January,Friday,13/01/2023,False,None,2024-11-10,False


In [8]:
dim_fecha.to_sql('dim_fecha', etl_conn, if_exists='replace',index_label='key_dim_fecha')

723